In [1]:
import pandas as pd
import os

In [2]:
#there are other two extra files in the folder we don't need
excel_to_merge = ['data/{}'.format(x) for x in os.listdir('data')[:-3]]

In [3]:
#initialise the empty df
final = pd.DataFrame(columns=['Year', 'Gender', 'Age_Bands', 'Population'])

In [4]:
for path in excel_to_merge:
    df = pd.read_excel(path)
    #'cause someone forgot to remove footnotes!
    df = df.iloc[:-5,]
    mask = df['Data Zone'].str.contains('S0100234[1-7]')
    age_cols = df.columns.values[3:]
    temp = df[mask].melt(id_vars=['Data Zone', 'Year', 'Gender'], value_vars=age_cols,
                       var_name='Age_Bands', value_name='Population')
    temp.loc[:,'Age_Bands'] = temp['Age_Bands'].str.strip()
    temp = (temp.groupby(['Year', 'Gender', 'Age_Bands'])['Population']
         .sum()
         .reset_index(name='Population'))
     
    final = pd.concat([final, temp], sort=True)
    print('done')

done
done
done
done


In [5]:
final['Area'] = 'Uist'
final['Year'] = final['Year'].astype('int').astype('str')
final.replace({'Age_Bands':{'90 & over':'90+'}}, inplace=True)

In [6]:
#add uist 2016 data - bearing in mind that the age bands are different.
df_2016 = (pd.read_csv('data/pyramid_data.csv')
            .groupby(['Age_Bands', 'Area', 'Gender'])['Population']
            .sum()
            .reset_index(name='Population'))

In [7]:
df_2016 = df_2016[df_2016['Area'] == 'Uist']

In [8]:
df_2016['Year'] = '2016'

In [10]:
final = pd.concat([final, df_2016], sort=True)

In [11]:
final.to_csv('data/uist_time_pop_pyramid.csv', index=False)

#### Clean up function for data downloaded from statistics.gov.scot

In [ ]:
def data_merge(df_male, df_female, year):
    df_m = df_male.copy()
    df_f = df_female.copy()
    
    def data_clean(dataframe, gender):
        df = dataframe.copy()
        df['DZ'] = df.iloc[:,0].str[52:]
        mask = df['DZ'].str.contains('S0100234[1-7]')
        df = df[mask].iloc[:,6:].assign(Gender=gender)
        return df
    
    df_m = data_clean(df_m, 'Male')
    df_f = data_clean(df_f, 'Female')
    
    df = pd.concat([df_m, df_f])
    df['Year'] = year
    age_cols = df.columns.values[:-3]
    df = (df.melt(id_vars=['DZ', 'Year', 'Gender'], value_vars=age_cols,
                    var_name='Age_Bands', value_name='Population')
            .groupby(['Year', 'Gender', 'Age_Bands'])['Population']
                    .sum()
                    .reset_index(name='Population')
            .replace({'Age_Bands':{'90 And Over':'90+'}}))
    
    return df

In [ ]:
m = pd.read_csv(r'C:\Users\germap01\Desktop\data\2009_male.csv',skiprows=8)
f = pd.read_csv(r'C:\Users\germap01\Desktop\data\2009_female.csv',skiprows=8)
data_merge(m, f, '2009').to_csv('2009_clean.csv', index=False)